In [1]:
# 解压数据和模型到工作目录
# !pip install paddle-gpu==2.4.0rc0
# !tar zxvf /home/aistudio/data/data174964/ernie_dqa_task1.tar.gz -C /home/aistudio/work
#预训练模型解压到指定目录
# !tar zxvf /home/aistudio/data/data178578/ernie_3.0_x_base_ch.tar.gz -C /home/aistudio/work/ernie_dqa_task1/applications/models_hub/
!tar zxvf /home/aistudio/data/data181867/cls_embedding.txt.tar.gz -C /home/aistudio/work/ernie_dqa_task1_ner_crf_sterp2/applications/models_hub

./cls_embedding.txt


1.
    赛道一的任务一最终提交结果的三个方法分别为利用ner方式预测但没有使用beam-search的模型/work/ernie_dqa_task1_ner，以及使用了beam-search的实体识别方法/work/ernie_dqa_task1_ner_crf，以及分段阅读理解的模型/work/ernie_dqa_task1_seg_attention；

In [3]:
#1.1 只进行了ner数据标注，而没有使用crf的训练模型训练;/home/aistudio/work/ernie_dqa_task1_ner
%cd /home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling
# 执行训练任务脚本
!sh begin_train.sh

/home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling
INFO: 01-03 17:22:22: params.py:43 * 140163065014016 ./examples/seqlab_ernie_fc_ch.json
INFO: 01-03 17:22:22: params.py:52 * 140163065014016 {
    "dataset_reader": {
        "dev_reader": {
            "config": {
                "batch_size": 8,
                "data_path": "./data/dev_data",
                "epoch": 1,
                "need_data_distribute": true,
                "need_generate_examples": false,
                "sampling_rate": 1.0,
                "shuffle": false
            },
            "fields": [
                {
                    "data_type": "string",
                    "embedding": null,
                    "max_seq_len": 512,
                    "name": "text_a",
                    "need_convert": true,
                    "padding_id": 0,
                    "reader": {
                        "type": "ErnieTextFieldReader"
                    },
                    "tokeni

In [2]:
#1.2.1 ner没有crf的模型推理,主要是把test数据集的doc_text按照512分割，以id为同一个文档为唯一标识，为了后来把同一文档答案合并做唯一标志 分割后9631个样本
#   在预测时需要修改/home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling/examples/seqlab_ernie_fc_ch_infer.json的
#   inference_model_path对应模型保存路径 最终测试数据0.674
%cd /home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling
!python run_infer.py --param_path=./examples/seqlab_ernie_fc_ch_infer.json
#   预测结果保存在/home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling/output/4001.txt
#1.2.2 合并为最终提交结果,最终结果保存在/home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling/output/subtask1_test_pred_ner.txt
!sh final_submit.sh
# 评测开发集指标
# !sh evaluate.sh

/home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling
finished construct submit files.............


In [5]:
#2.1 进行了ner数据标注，同时使用crf的训练模型训练  /home/aistudio/work/ernie_dqa_task1_ner_crf
%cd /home/aistudio/work/ernie_dqa_task1_ner_crf/applications/tasks/sequence_labeling
# 执行训练任务脚本
!sh begin_train.sh

/home/aistudio/work/ernie_dqa_task1_ner_crf/applications/tasks/sequence_labeling
INFO: 01-03 18:54:57: params.py:43 * 140335628420864 ./examples/seqlab_ernie_fc_ch.json
INFO: 01-03 18:54:57: params.py:52 * 140335628420864 {
    "dataset_reader": {
        "dev_reader": {
            "config": {
                "batch_size": 8,
                "data_path": "./data/dev_data",
                "epoch": 1,
                "need_data_distribute": true,
                "need_generate_examples": false,
                "sampling_rate": 1.0,
                "shuffle": false
            },
            "fields": [
                {
                    "data_type": "string",
                    "embedding": null,
                    "max_seq_len": 512,
                    "name": "text_a",
                    "need_convert": true,
                    "padding_id": 0,
                    "reader": {
                        "type": "ErnieTextFieldReader"
                    },
                    "to

In [3]:
#2.2.1 进行了ner数据标注，同时使用beamsearch进行推理,主要是把test数据集的doc_text按照512分割，以id为同一个文档为唯一标识，为了后来把同一文档答案合并做唯一标志 
#   分割后9631个样本;
#   在预测时需要修改/home/aistudio/work/ernie_dqa_task1_ner_crf/applications/tasks/sequence_labeling/examples/seqlab_ernie_fc_ch_infer.json的
#   inference_model_path对应模型保存路径  最终测试数据集分数0.679;
%cd /home/aistudio/work/ernie_dqa_task1_ner_crf/applications/tasks/sequence_labeling
!python run_infer.py --param_path=./examples/seqlab_ernie_fc_ch_infer.json
#   预测结果保存在/home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling/output/4001.txt
#2.2.2 合并为最终提交结果,最终提交结果文件路径在./output/subtask1_test_pred_ner_crf.txt

!sh final_submit.sh
# 评测开发集指标
# !sh evaluate.sh

/home/aistudio/work/ernie_dqa_task1_ner_crf/applications/tasks/sequence_labeling
finished construct submit files.............


In [8]:
#3.1 使用crf的训练模型训练,同时对passage分为两段进行attention进行训练模型 /home/aistudio/work/ernie_dqa_task1_seg_attention
%cd /home/aistudio/work/ernie_dqa_task1_seg_attention/applications/tasks/sequence_labeling
# 执行训练任务脚本
!sh begin_train.sh

/home/aistudio/work/ernie_dqa_task1_seg_attention/applications/tasks/sequence_labeling
INFO: 01-03 19:09:55: params.py:43 * 140548029671168 ./examples/seqlab_ernie_fc_ch.json
INFO: 01-03 19:09:55: params.py:52 * 140548029671168 {
    "dataset_reader": {
        "dev_reader": {
            "config": {
                "batch_size": 8,
                "data_path": "./data/dev_data",
                "epoch": 1,
                "need_data_distribute": true,
                "need_generate_examples": false,
                "sampling_rate": 1.0,
                "shuffle": false
            },
            "fields": [
                {
                    "data_type": "string",
                    "embedding": null,
                    "max_seq_len": 512,
                    "name": "text_a",
                    "need_convert": true,
                    "padding_id": 0,
                    "reader": {
                        "type": "ErnieTextFieldReader"
                    },
                 

KeyboardInterrupt: 

In [5]:
#3.2.1 使用crf的训练模型训练,同时对passage分为两段进行attention进行推理,主要是把test数据集的doc_text按照1024分割，以id为同一个文档为唯一标识，为了后来把同一文档答案合并做唯一标志 
#      分割后6100个样本
#      在预测时需要修改/home/aistudio/work/ernie_dqa_task1_ner_crf/applications/tasks/sequence_labeling/examples/seqlab_ernie_fc_ch_infer.json的
#      inference_model_path对应模型保存路径  最终测试数据集分数0.68025
%cd /home/aistudio/work/ernie_dqa_task1_seg_attention/applications/tasks/sequence_labeling
!python run_infer.py --param_path=./examples/seqlab_ernie_fc_ch_infer.json
#预测结果保存在/home/aistudio/work/ernie_dqa_task1_ner/applications/tasks/sequence_labeling/output/40001.txt
#3.2.2 合并为最终提交结果,最终提交结果文件路径在./output/subtask1_test_pred_seg.txt
!sh final_submit.sh
# 评测开发集指标
# !sh evaluate.sh

/home/aistudio/work/ernie_dqa_task1_seg_attention/applications/tasks/sequence_labeling
finished construct submit files.............


In [7]:
#4.1   最终把这三个模型最终结果进行投票融合
%cd /home/aistudio/work
!python combine.py

/home/aistudio/work
finished combine submit files..............................
